In [1]:
import pandas as pd
import json
import random

In [2]:
# Load entity_params into a dictionary

print("Started Reading JSON file")
with open("entity_params.json", "r") as read_file:
    print("Converting JSON encoded data into Python dictionary")
    entity_params_dict = json.load(read_file)

Started Reading JSON file
Converting JSON encoded data into Python dictionary


In [3]:
# set paramters
num_biz = 1000
criminal_biz_proportion = 0.01 # proportion of criminal businesses

In [4]:
# Create num_biz business ids
entity_id = ["B_" + str(x) for x in range(1, num_biz + 1)]

In [5]:
# Create business region by sampling from region distribution in param dict
total_biz = sum(entity_params_dict["business"]["region_dist"].values())
region_list = list(entity_params_dict["business"]["region_dist"].keys())
region_weights = list(entity_params_dict["business"]["region_dist"].values())

region = random.choices(region_list, weights=region_weights, k=num_biz)

In [9]:
# Add in lad17 code
lad17cd = list(map(entity_params_dict["business"]["region_lad_map"].get, region))

In [10]:
# Create business sector by sampling from conditional distribution of sector
# given region from param dict

sector = []
for rgn in region:
    sector_list = list(entity_params_dict["business"]["region_sector_dist"][rgn].keys())
    sector_weights = list(entity_params_dict["business"]["region_sector_dist"][rgn].values())
    sector.append(random.choices(sector_list, weights = sector_weights, k=1)[0])
    

In [11]:
# Create business sector by sampling from conditional distribution of turnover
# given region from param dict

turnover = []
for rgn in region:
    turnover_list = list(entity_params_dict["business"]["region_turnover_dist"][rgn].keys())
    turnover_weights = list(entity_params_dict["business"]["region_turnover_dist"][rgn].values())
    turnover.append(random.choices(turnover_list, weights = turnover_weights, k=1)[0])

turnover = [x.replace('+','-1000000') for x in turnover]
turnover = [x.split('-') for x in turnover]
turnover = [int(random.uniform(int(x[0]),int(x[1]))) * 1000 for x in turnover]

In [12]:
# Create criminal flag
criminal = random.choices([0,1], weights = [1-criminal_biz_proportion, criminal_biz_proportion], k=num_biz)

In [13]:
# Create account ids, for now each business has a current and a loan account
# account ids are just the entity_id with the suffic CUR or LOA

account_ids = []
for entity in entity_id:
    account_ids.append([str(entity) + '_CUR', str(entity) + '_LOA'])

In [14]:
business_df = pd.DataFrame({
    "entity_id" : entity_id,
    "region" : region,
    "lad17cd": lad17cd,
    "sector" : sector,
    "turnover" : turnover,
    "account_ids" : account_ids,
    "criminal" : criminal
})

In [15]:
business_df.head()

,entity_id,region,lad17cd,sector,turnover,account_ids,criminal
0,B_1,East Midlands,E12000004,Accommodation food services,226000,"[B_1_CUR, B_1_LOA]",0
1,B_2,Scotland,S92000003,Construction,123000,"[B_2_CUR, B_2_LOA]",0
2,B_3,North West,E12000002,Business administration support services,17337000,"[B_3_CUR, B_3_LOA]",0
3,B_4,South East,E12000008,Business administration support services,7530000,"[B_4_CUR, B_4_LOA]",0
4,B_5,South East,E12000008,Production,61000,"[B_5_CUR, B_5_LOA]",0
